In [1]:
#####################################   Importing data   #########################################################################################

import numpy as np
from vpython import *
from ipywidgets import interactive
from IPython.display import clear_output
from playsound import playsound
import _thread
import time

###############################################################################################################################

<IPython.core.display.Javascript object>

In [2]:
# mixing factors (converted from degrees to radians)
th12 = np.radians(33.82)
th23 = np.radians(48.3)
th13 = np.radians(8.61)
delta_CP = np.radians(222)
#delta_CP = 0

# mass differences
dm21_sq = 7.39*10**(-5) # eV^2
dm31_sq = 2.523*10**(-3) # eV^2

# mass eigenstates
m1 = 0.01 # eV
m2 = np.sqrt(m1*m1 + dm21_sq) # eV
m3 = np.sqrt(m1*m1 + dm31_sq) # eV 

# total mass 
mtot = m1 + m2 + m3 #eV
assert mtot < 0.23, "Error: total mass not in agreement with experimental bounds"

# calculate third mass difference squared
dm32_sq = m3*m3 - m2*m2 # eV^2

# set energy values
E_sun_pp = 4*10**5 # eV 
E_atm = 10**11 # eV
E_acc = 10**9 #eV

# important constants:

# speed of light
c = 299792.458 # km s^-1 

# Planck constant
hbar = 6.582119569*10**(-16) # eV*s

# Astronomical unit (i.e. the distance between the Sun and the Earth)
AU = 149597870.700 # km

# Equatorial radius of the Sun
r_sun = 696342 # km
r_sun_core = 0.12*r_sun # km
# Volume sun and core
V_sun = 4*np.pi*r_sun*r_sun*r_sun / 3 # km^3
V_sun_core = 4*np.pi*r_sun_core*r_sun_core*r_sun_core / 3 # km^3
V_rest = V_sun - V_sun_core # km^3

# Electron density
N0 = 6.02214076*10**23 # Avogadro's number
mean_density_sun_core = 113*10**3 # kg m^-3
mean_mol_weight_e = 1.69 
e_density_sun_core = N0 * mean_density_sun_core / mean_mol_weight_e

# mass eigenstates vector
mar_3f = np.array([m1, m2, m3]) 

#rotational matrix
U1 = np.array([[1, 0, 0], [0, np.cos(th23), np.sin(th23)], [0, -np.sin(th23), np.cos(th23)]]) #first matrix
U2 = np.array([[np.cos(th13), 0, np.sin(th13)*np.exp(-1j*delta_CP)], [0, 1, 0], 
               [-np.sin(th13)*np.exp(1j*delta_CP), 0, np.cos(th13)]]) #second matrix
U3 = np.array([[np.cos(th12), np.sin(th12), 0], [-np.sin(th12), np.cos(th12), 0], [0, 0, 1]]) #third matrix
rot_mat_3f = np.dot(U1, np.dot(U2, U3)) #final matrix

# check that the 3x3 matrix is unitary
herm = np.transpose(np.conjugate(rot_mat_3f))

########

def full_prob(L, E, alpha):
    """Calculates the probability of a neutrino of flavour alpha to remain of that same flavour or to change into another
    flavour
    Takes as inputs the distance L (in kilometers), the energy at which the neutrino is created E (in eV), 
    and the initial flavour alpha ("electron", "muon", or "tau")
    Returns an array with three elements respectively representing the probability of the neutrino being an electron, muon
    or tau neutrino"""
    
    # transform from string to integer
    if alpha == "electron":
        a = 0
    elif alpha == "muon":
        a = 1
    elif alpha == "tau":
        a = 2
    else:
        print('Error: choose from "electron", "muon" or "tau"')
        
    # calculate X_{ij} (argument of sine squared)
    X = np.zeros((3, 3)) #store
    for i in range(3):
        for j in range(3):
            X[i, j] = 1267000000*(mar_3f[i]*mar_3f[i] - mar_3f[j]*mar_3f[j]) * L / (4*E)
        
    # calculate probabilities
    U = rot_mat_3f 
    vals = np.zeros(3) #store
    for b in range(3): 
        sum_re_parts = 0
        sum_im_parts = 0
        for i in range(3):
                for j in range(i):
                    term = np.conj(U[a, i])*U[b, i]*U[a, j]*np.conj(U[b, j])
                    re_part = np.real(term)*np.sin(X[i,j])*np.sin(X[i,j])
                    im_part = np.imag(term)*np.sin(2*X[i,j])
                    sum_re_parts += re_part
                    sum_im_parts += im_part
        if b == a:
            vals[b] = 1 - 4*sum_re_parts + 2*sum_im_parts
        else:
            vals[b] = - 4*sum_re_parts + 2*sum_im_parts
            
    return vals

In [3]:
####### Setting up initial data #######

g = 35               # gravitational acceleration (m.s^-2)
mass_bird = 1        # mass of the bird (kg)
radius_bird = 0.2    # radius of the bird (m)
x0 = 0.0             # initial x-coordinate of the bird
y0 = 0.5 + radius_bird # initial y-coordinate of the bird
p = 300              # density of the target (kg.m^-3) ~ density of a balsa type of wood
contact_time = 0.01  # contact time of the impact between the bird and the target (sec)
dt = 0.012           # time interval for the animations (sec)
d_angle = 0.06       # angle interval for the animation of the toppling of the target (rad)

bird_color = [color.red,color.blue,color.green]
bird_mass_topple = [0.9,2.2,1.5]

empty_heart = "\u2661" + " "
full_heart = "\u2665" + "  "

V_center = vector(8,0,0)
V_camera = vector(4,2,12)
V_title = vector(2,6,-2)
V_button = vector(-1,1,2)

In [4]:
####################### Functions ###########################

def trajectory(x_pos):
    '''
    Calculates the trajectory (y coordinate) at a given horizontal distance (x coordinate) from the lauch point.
    
    Input: 
     - x_pos: x coordinate of the point where the trajectory is required.
     
    Output:
     - trajectory: y coordinate of the particle at this point.
    '''
    
    trajectory = -g*(x_pos-x0)**2 / (2*v0**2*np.cos(theta)**2) + np.tan(theta)*(x_pos-x0) + y0
    
    return trajectory    



def vector_list():   
    global v0
    global theta
    Vectors=[]
    xpoints_slider=np.linspace(0,3,15)
    for x in xpoints_slider:
        Vectors.append(vector(x,trajectory(x),0))
    return Vectors

def setV(v):
    global v0
    wt1.text = 'Launch speed: ' + '{:1.0f}'.format(sl1.value*1.25) + ' km/s'
    v0 = v.value
    c.clear()
    c.append(vector_list())
    
def setTheta(t):
    global theta
    wt2.text = 'Launch angle: ' + '{:1.0f}'.format(sl2.value) + ' degrees'
    theta = np.radians(t.value)
    c.clear()
    c.append(vector_list())

In [12]:
######################### ANIMATION #####################

def runanim():
    
    global scene
    global v0
    global x0
    global y0
    global mass_topple
    global theta
    global target
    global c
    global level
    global life
    global label_level
    global highscore
    global highscore_label
    global target_pos
    global sl1
    global sl2
    
    # Initial data with respect to the scene set up 
    mass_target = p*(target.length*target.width*target.height)  # setting up the mass of the target (m = p*V)
    hit_tolerance = radius_bird + target.length/2               # computing the value of the hit tolerance : x-distance between bird.pos and target.pos at the time of the collision
    x_impact = target.pos.x - hit_tolerance                     # x-coordinate of the impact point
    t_restoring = mass_target*g*target.length/2                 # computing the magnitude of the restoring torque of the target
    
    obj = scene.mouse.pick
    
    if (obj == launch_button) :
        
        scene.unbind('click', runanim)
        sl1.disabled = True
        sl2.disabled = True
        
        c.clear()
        
        _thread.start_new_thread (playsound, ("Launch.mp3",))
        time.sleep(0.5)

        x = x0
        y = y0
        t = 0
        distance_covered = 0

        # while the ball hasn't reached the target
        while x <= x_impact and y > 0 :

            # time increases by dt
            rate(25)
            t += dt

            distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

            prob = full_prob(1.25*distance_covered, E_sun_pp, 'tau').tolist()
            prob_max = np.amax(prob)
            index_max = prob.index(prob_max)

            mass_topple = bird_mass_topple[index_max]
            bird.color = bird_color[index_max]
            bird.trail_color = bird_color[index_max]

            x = x0 + v0*t*np.cos(theta)               # new x-position of the bird
            y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2  # new y-position of the bird
            bird.pos = vector(x,y,0)                  # updating the position of the bird


        # if the bird misses with the target
        if y > target.height + bird.radius/2 :

            # while the bird is above the ground 
            while x < 35 and y >= 0:

                # time increases by dt
                rate(25)
                t += dt

                distance_covered += np.sqrt(abs(x - (x0 + v0*t*np.cos(theta)))**2 + abs(y - (y0 + v0*t*np.sin(theta) - 0.5*g*t**2))**2)

                prob = full_prob(1.25*distance_covered, E_sun_pp, 'tau').tolist()
                prob_max = np.amax(prob)
                index_max = prob.index(prob_max)
                
                mass_topple = bird_mass_topple[index_max]
                bird.color = bird_color[index_max]
                bird.trail_color = bird_color[index_max]

                x = x0 + v0*t*np.cos(theta)               # new x-position of the bird
                y = y0 + v0*t*np.sin(theta) - 0.5*g*t**2  # new y-position of the bird
                bird.pos = vector(x,y,0)                  # updating the position of the bird
    

        # if the birds collides with target
        elif 0 < y < target.height + bird.radius/2 :
            
            _thread.start_new_thread (playsound, ("Boing.mp3",))

            c_rot = target.pos + vector(target.length/2,-target.height/2,0)   # center of rotation position
            bird_momentum = vector(mass_topple*v0*np.cos(theta) , mass_topple*v0*np.sin(theta) - mass_topple*g*t , 0) # momentum of the bird at the time of the impact
            vector_da = vector(x_impact,trajectory(x_impact),0) - c_rot       # vector from the point of rotation to the point of impact
            t_applied_vector = cross(bird_momentum/contact_time,vector_da)    # torque applied by the bird on the target
            t_applied = mag(t_applied_vector)                                 # magnitude of the torque applied 

            # if the applied torque is enough to topple the target 
            if t_applied > t_restoring :
                
                v = vector(x0 + v0*(t+1)*np.cos(theta) - x, y0 + v0*(t+1)*np.sin(theta) - 0.5*g*(t+1)**2 - y,0)
                
                t_bounce = 0 
                x0_bounce = x
                y0_bounce = y
                v0_bounce = 0.35 * mag(v)
                theta_bounce = np.arctan(v.y/v.x)
                
                dtheta=0
                t=0
    
                while dtheta<=(np.pi/2):
            
                    rate(25)
                    dtheta+=(np.pi/2+0.002)/20
                    target.rotate(angle=(np.pi/2+0.002)/20,axis=vector(0,0,-1),origin=vector(target_pos+target.length/2,0,0))
                    
                    if y > bird.radius :
                        
                        t_bounce += dt  # time increases by dt                    
                        
                        distance_covered += np.sqrt(abs(x - (x0_bounce - v0_bounce*t_bounce*np.cos(theta_bounce)))**2 + abs(y - (y0_bounce + v0_bounce*t_bounce*np.sin(theta_bounce) - 0.5*g*t_bounce**2))**2)

                        prob = full_prob(1.25*distance_covered, E_sun_pp, 'tau').tolist()
                        prob_max = np.amax(prob)
                        index_max = prob.index(prob_max)

                        mass_topple = bird_mass_topple[index_max]
                        bird.color = bird_color[index_max]
                        bird.trail_color = bird_color[index_max]
                        
                        x = x0_bounce - v0_bounce*t_bounce*np.cos(theta_bounce)               # new x-position of the bird
                        y = y0_bounce + v0_bounce*t_bounce*np.sin(theta_bounce) - 0.5*g*t_bounce**2  # new y-position of the bird  

                        bird.pos = vector(x,y,0)    # updating the position of the bird
                
                level+=1
                target.height = 3 + 1.5 * random()
                target_pos = 5 + level*2
                target.pos = vector(target_pos,target.height/2,0)
                target.axis = vector(1,0,0)
                label_level.text= "Level " + str(level)
                
                life += 1
                
                if level > highscore :
                    highscore += 1
                    highscore_label.text = "Highscore: Level  " + str(highscore)
                 
                    
            # if the applied torque isn't enough to topple the target --> falling of the bird following a vertical straight line
            else :
                
                v = vector(x0 + v0*(t+1)*np.cos(theta) - x, y0 + v0*(t+1)*np.sin(theta) - 0.5*g*(t+1)**2 - y,0)
                
                t_bounce = 0 
                x0_bounce = x
                y0_bounce = y
                v0_bounce = 0.35 * mag(v)
                theta_bounce = np.arctan(v.y/v.x)
                
                # while the bird is still above the ground 
                while y > bird.radius :
                    
                    rate(25)
                    
                    t_bounce += dt  # time increases by dt                    
                        
                    distance_covered += np.sqrt(abs(x - (x0_bounce - v0_bounce*t_bounce*np.cos(theta_bounce)))**2 + abs(y - (y0_bounce + v0_bounce*t_bounce*np.sin(theta_bounce) - 0.5*g*t_bounce**2))**2)

                    prob = full_prob(1.25*distance_covered, E_sun_pp, 'tau').tolist()
                    prob_max = np.amax(prob)
                    index_max = prob.index(prob_max)

                    mass_topple = bird_mass_topple[index_max]
                    bird.color = bird_color[index_max]
                    bird.trail_color = bird_color[index_max]
                        
                    x = x0_bounce - v0_bounce*t_bounce*np.cos(theta_bounce)               # new x-position of the bird
                    y = y0_bounce + v0_bounce*t_bounce*np.sin(theta_bounce) - 0.5*g*t_bounce**2  # new y-position of the bird  

                    bird.pos = vector(x,y,0)    # updating the position of the bird
    
        
        # Updating lifes
        life += -1
        
        i = 1
        life_text = " Lives    -    "
        while i <= life:
            life_text = life_text + full_heart
            i += 1
        while i <= 5:
            life_text = life_text + empty_heart
            i += 1
            
        # Initialisation
        if bird.make_trail == True :
            bird.make_trail = False
            
        life_label.text = life_text
        bird.clear_trail()
        bird.pos = vector(x0,y0,0)
        bird.color = bird_color[2]
        mass_topple = bird_mass_topple[2]
        bird.make_trail = True
        
        c.append(vector_list())
        
        while bird.pos != vector(x0,y0,0):
            t = 0
            
        scene.bind('click', runanim)
        sl1.disabled = False
        sl2.disabled = False
          
        # Game over action
        if life <= 0:
            
            time.sleep(1)
            _thread.start_new_thread (playsound, ("GameOver.mp3",))
            
            game_over = label(text = 'GAME OVER', pos = vector(6,2,2), color=vector(0.7,0.7,0.7) , height = 10, box = False, opacity = 0)
            t = 0
            while t <= 5000:
                rate(25)
                game_over.height += 1
                t += 50
                
            scene.delete()
            clear_output()
            game_start()

#############################################################################################################################

In [13]:
################### CANVAS #######################

def game_start() :
    
    global scene
    global v0
    global theta
    global target
    global c
    global level
    global life
    global label_level
    global life_label
    global highscore
    global highscore_label
    global target_pos
    global target_height
    global launch_button
    global sl1
    global sl2
    global bird
    global wt1
    global wt2
    
    # Initial parameters
    v0=15
    theta=45
    target_height = 3 + 1.5*random()
    level = 1
    target_pos= 5 + 2*level
    life = 5

    # Setting up the scene 
    print('\n\n\n\n\n')
    scene = canvas(width=1000, height=650, center=V_center, range=8, background=color.cyan)   # scene where the game animations will be displayed
    scene.camera.pos = V_camera                                                               # position of the camera (from where the player sees the game)
    scene.forward = V_center - scene.camera.pos                                               # axis of the camera (in which direction the player is looking)
    scene.camera.axis = V_center - scene.camera.pos                                           # axis of the camera (in which direction the player is looking)
    scene.userzoom = False
    scene.userspin = False

    # positioning the ground of the animation
    i = -50
    j = -50
    while i <= 100:
        while j <= 50:
            box(pos = vector(i,-0.01,j), length=5, height=0.01, width=5, texture = 'grass.jpg')  # texture = '/nbextensions/vpython_data/grass.jpg')
            j += 5
        j = -50
        i += 5

    # positioning the target at a random position
    target = box(pos=vector(target_pos,target_height/2,0), length=0.5, height=target_height , width=0.5, color = color.black)

    # box
    pedestal = box(pos = vector(0,0.25,0), length = 1, width = 2, height = 0.5, texture = textures.wood)

    # Sky
    sky = box(pos=vector(50,22,-50),axis=vector(1,0,0.7), length=200, width=1, height=50, texture = 'Clouds.jpg')  # texture = '/nbextensions/vpython_data/Clouds.jpg')
    
    # Legend 
    legend = label(text = '    Mass  Hierarchy    \n\n >         > \n', pos = vector(25,9,0), color = color.black,  box = False, opacity = 0, height = 23) 
    legend_muon = sphere(pos = vector(20,6.6,0), radius = 0.7, color=color.blue)
    legend_tau = sphere(pos = vector(25.6,7,0), radius = 0.6, color=color.green)
    legend_elec = sphere(pos = vector(32.8,7.5,0), radius = 0.5, color=color.red)

    # positioning the bird on the origin of the scene
    bird = sphere(pos = vector(x0,y0,0), radius = 0.2, color=color.green, make_trail = True, trail_color = color.green, trail_radius = 0.035, pps = 5)

    # Text
    label_title = label(text = 'ANGRY NEUTRINOS',pos = vector(8,6.9,0), color = color.black, box = False , height = 40, opacity = 0)
    label_level = label(text = "Level " + str(level) , pos = vector(8,5.9,0), color = color.black, box = False, height = 35, opacity = 0)

    # Launch button
    launch_button = box(pos = V_button, length = 0.1, width = 0.3, height = 0.3, axis = V_button + V_camera + vector(-4,0,0), color = color.black)
    launch_button.rotate(angle =- np.pi/6.5, axis= V_camera, origin=V_button)
    label_button = label(text = 'GO',pos = V_button + vector(0,0,-0.05), color = color.white, box = False, background = color.black , height = 25, opacity = 1, border = 10)

    # Lives
    life_text = " Lives    -    " + full_heart + full_heart + full_heart + full_heart + full_heart
    life_label = label(text = life_text, pos = vector(4.25, 0, 9.25) , height = 30 , color = color.red , box = False, opacity = 0)

    # Highscore
    highscore_label = label(text = "Highscore: Level  " + str(highscore), pos = vector(1,6,0), color = color.black , box = False, opacity = 0, height = 23)
    
    # Scale
    scale = curve (vector(-0.8,0.5,6),vector(0.2,0.5,6), color = color.black)
    scale_start = box(pos = vector(-0.8,0.5,6), length = 0.05, width = 0.1, height = 0.1, color = color.black )
    scale_end = box(pos = vector(0.2,0.5,6), length = 0.05, width = 0.1, height = 0.1, color = color.black )
    scale_label = label(text = '1.25 km', pos = vector(-0.1,0.5,6.55), color = color.black , box = False, opacity = 0, height = 23)
    
    # Aiming
    c = points(radius = 2.5)  
    c.append(vector_list())

    # Slider Speed
    sl1 = slider(min = 10 , max = 30 , value = 15, length = 220 , bind = setV, left = 50, right = 15)
    wt1 = wtext(text = 'Launch speed: ' + '{:1.0f}'.format(sl1.value*1.25) + ' km/s') 

    # Slider angle    
    sl2 = slider(min = 0 , max = 90 , value = 45, length = 220 , bind = setTheta, left = 150, right = 15)
    wt2 = wtext(text = 'Launch angle: ' + '{:1.0f}'.format(sl2.value) + ' degrees') 

    # Launch by clicking on animation
    scene.bind('click', runanim)
    
    print('\n\n\n\n\n')
    
##################################################################################################################################################################################

<IPython.core.display.Javascript object>

In [11]:
highscore = 1
game_start()

<IPython.core.display.Javascript object>